<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/agent/openai_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# 构建你自己的OpenAI代理


使用支持函数调用的[新OpenAI API](https://openai.com/blog/function-calling-and-other-api-updates)，构建自己的代理程序变得前所未有的简单！

在这个笔记本教程中，我们展示了如何在**不到50行代码**的情况下编写自己的OpenAI代理程序！它非常简洁，但功能齐全（具有进行对话和使用工具的能力）。


## 初始设置


让我们从导入一些简单的基本模块开始。

我们主要需要的是：
1. OpenAI API（使用我们自己的 `llama_index` LLM 类）
2. 保存对话历史的位置
3. 我们的代理可以使用的工具的定义。


如果您在Colab上打开这个笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
%pip install llama-index-agent-openai
%pip install llama-index-llms-openai

In [ ]:
!pip install llama-index

In [ ]:
import json
from typing import Sequence, List

from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
from openai.types.chat import ChatCompletionMessageToolCall

import nest_asyncio

nest_asyncio.apply()

让我们为我们的代理人定义一些非常简单的计算器工具。


In [ ]:
def multiply(a: int, b: int) -> int:
    """将两个整数相乘，并返回结果整数"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

In [ ]:
def add(a: int, b: int) -> int:
    """对两个整数进行相加，并返回结果整数"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

## 代理（Agent）定义


现在，我们定义一个能够进行对话并在**不到50行代码**中调用工具的代理程序。

代理程序逻辑的核心在于`chat`方法。在高层次上，有3个步骤：
1. 调用OpenAI来决定是否调用工具以及使用什么参数。
2. 使用参数调用工具以获得输出。
3. 调用OpenAI根据对话上下文和工具输出来合成一个回复。

`reset`方法只是简单地重置对话上下文，这样我们就可以开始另一个对话。


In [ ]:
class YourOpenAIAgent:
    def __init__(
        self,
        tools: Sequence[BaseTool] = [],
        llm: OpenAI = OpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
        chat_history: List[ChatMessage] = [],
    ) -> None:
        self._llm = llm
        self._tools = {tool.metadata.name: tool for tool in tools}
        self._chat_history = chat_history

    def reset(self) -> None:
        self._chat_history = []

    def chat(self, message: str) -> str:
        chat_history = self._chat_history
        chat_history.append(ChatMessage(role="user", content=message))
        tools = [
            tool.metadata.to_openai_tool() for _, tool in self._tools.items()
        ]

        ai_message = self._llm.chat(chat_history, tools=tools).message
        additional_kwargs = ai_message.additional_kwargs
        chat_history.append(ai_message)

        tool_calls = additional_kwargs.get("tool_calls", None)
        # 现在支持并行函数调用
        if tool_calls is not None:
            for tool_call in tool_calls:
                function_message = self._call_function(tool_call)
                chat_history.append(function_message)
                ai_message = self._llm.chat(chat_history).message
                chat_history.append(ai_message)

        return ai_message.content

    def _call_function(
        self, tool_call: ChatCompletionMessageToolCall
    ) -> ChatMessage:
        id_ = tool_call.id
        function_call = tool_call.function
        tool = self._tools[function_call.name]
        output = tool(**json.loads(function_call.arguments))
        return ChatMessage(
            name=function_call.name,
            content=str(output),
            role="tool",
            additional_kwargs={
                "tool_call_id": id_,
                "name": function_call.name,
            },
        )

## 让我们试一试！


In [ ]:
agent = YourOpenAIAgent(tools=[multiply_tool, add_tool])

In [ ]:
agent.chat("Hi")

'Hello! How can I assist you today?'

In [ ]:
agent.chat("What is 2123 * 215123")

'The product of 2123 multiplied by 215123 is 456,706,129.'

## 我们（稍微改进的）`OpenAIAgent` 实现


我们在LlamaIndex中提供了一个（稍微更好的）`OpenAIAgent`实现，您可以直接按照以下方式使用。

与上面的简化版本相比：
* 它实现了`BaseChatEngine`和`BaseQueryEngine`接口，因此您可以更无缝地在LlamaIndex框架中使用它。
* 它支持每次对话轮多个函数调用
* 它支持流式传输
* 它支持异步端点
* 它支持回调和追踪


In [ ]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-0613")
agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool], llm=llm, verbose=True
)

这是一个用Python编写的简单聊天程序。


In [ ]:
response = agent.chat("What is (121 * 3) + 42?")
print(str(response))

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363

STARTING TURN 2
---------------

=== Calling Function ===
Calling function: add with args: {
  "a": 363,
  "b": 42
}
Got output: 405

STARTING TURN 3
---------------

(121 * 3) + 42 is equal to 405.


In [ ]:
# 检查数据源
print(response.sources)

[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)]


### 异步聊天


In [ ]:
response = await agent.achat("What is 121 * 3?")
print(str(response))

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363

STARTING TURN 2
---------------

121 multiplied by 3 is equal to 363.


### 流式聊天
在这里，每个LLM响应都作为生成器返回。您可以流式传输每个增量步骤，或者只获取最后的响应。


In [ ]:
response = agent.stream_chat(
    "What is 121 * 2? Once you have the answer, use that number to write a"
    " story about a group of mice."
)

response_gen = response.response_gen

for token in response_gen:
    print(token, end="")

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 2
}
Got output: 242

STARTING TURN 2
---------------

121 multiplied by 2 is equal to 242.

Once upon a time, in a small village, there was a group of mice who lived in a cozy little burrow. The mice were known for their intelligence and resourcefulness. They had built a tight-knit community and worked together to overcome any challenges they faced.

One sunny day, as the mice were going about their daily activities, they stumbled upon a bountiful field of ripe corn. The field was filled with tall stalks of golden corn, swaying gently in the breeze. The mice couldn't believe their luck! They knew they had to gather as much corn as possible to sustain themselves through the upcoming winter.

With their tiny paws and sharp teeth, the mice began to harvest the corn. They worked tirelessly, carrying one ear of corn at a time back to their burrow. The mice were determined to 

### 异步流式聊天

这是一个简单的异步流式聊天示例，使用Python的asyncio库和websockets库。该示例包括一个简单的服务器和一个基本的Web客户端，用于在浏览器中进行聊天。

#### 服务器

服务器使用asyncio和websockets库创建一个简单的WebSocket服务器，用于接收和广播消息。

#### 客户端

客户端是一个简单的HTML页面，使用JavaScript和WebSocket API来连接到服务器并发送/接收消息。

#### 运行示例

要运行此示例，请按照以下步骤操作：

1. 安装所需的库：`pip install websockets`
2. 运行服务器：`python async_chat_server.py`
3. 在浏览器中打开`async_chat_client.html`，然后输入用户名并开始聊天。

现在，您可以在浏览器中与自己进行简单的异步流式聊天了！


In [ ]:
response = await agent.astream_chat(
    "What is 121 + 8? Once you have the answer, use that number to write a"
    " story about a group of mice."
)

response_gen = response.response_gen

async for token in response.async_response_gen():
    print(token, end="")

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: add with args: {
  "a": 121,
  "b": 8
}
Got output: 129

STARTING TURN 2
---------------

121 plus 8 is equal to 129.

Once upon a time, in a peaceful meadow, there lived a group of mice. These mice were known for their bravery and adventurous spirit. They loved exploring the meadow and discovering new places.

One sunny day, as the mice were scurrying through the tall grass, they stumbled upon a hidden treasure. It was a small, sparkling gemstone that radiated with a mesmerizing glow. The mice were amazed by its beauty and knew that it was something special.

Excitedly, the mice decided to take the gemstone back to their burrow. They carefully carried it, taking turns to ensure its safety. As they reached their cozy home, they marveled at the gemstone's brilliance. Little did they know, this gemstone held a magical power.

As the mice gathered around the gemstone, a soft, enchanting light began to emanate from

### 具有个性的代理

在这个示例中，我们将创建一个具有个性的代理，该代理将根据其个性特征做出不同的决策。我们将使用一个简单的示例来说明这个概念，该示例将根据代理的个性特征来选择不同的行为。

首先，让我们定义代理的个性特征。代理的个性特征将包括以下几个方面：
- 冒险程度（adventurousness）：代理愿意尝试新的行为或策略的程度。
- 谨慎程度（cautiousness）：代理对于采取行动时的谨慎程度。
- 礼貌程度（politeness）：代理与其他代理互动时的礼貌程度。

接下来，我们将根据代理的个性特征来定义其行为。例如，一个冒险程度高的代理可能更愿意尝试新的行为，而一个谨慎程度高的代理可能更倾向于选择较为保守的行为。

让我们通过一个简单的示例来说明这个概念。


您可以指定一个系统提示，以便为代理人提供额外的指示或个性。


In [ ]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.prompts.system import SHAKESPEARE_WRITING_ASSISTANT

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool],
    llm=llm,
    verbose=True,
    system_prompt=SHAKESPEARE_WRITING_ASSISTANT,
)

In [ ]:
response = agent.chat("Hi")
print(response)

STARTING TURN 1
---------------

Greetings, fair traveler! How may I assist thee on this fine day?


In [ ]:
response = agent.chat("Tell me a story")
print(response)

STARTING TURN 1
---------------

Of course, dear friend! Allow me to weave a tale for thee in the style of Shakespeare. 

Once upon a time, in a land far away, there lived a noble knight named Sir William. He was known throughout the kingdom for his bravery and chivalry. One fateful day, as Sir William rode through the enchanted forest, he stumbled upon a hidden glade.

In the glade, he discovered a beautiful maiden named Lady Rosalind. She was fair of face and gentle of heart, and Sir William was instantly captivated by her beauty. They spent hours conversing, sharing stories, and laughing together.

As the days turned into weeks, Sir William and Lady Rosalind's bond grew stronger. They found solace in each other's company and realized that they had fallen deeply in love. However, their love was not without obstacles.

Lady Rosalind's father, Lord Reginald, was a stern and overprotective man. He had already arranged a marriage for his daughter with a wealthy nobleman, Lord Percival. W